In [1]:
# do not touch this cell 
%load_ext jupyter_record


# Imports


In [14]:
%%git_commit

import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder


# Dataset Load




In [22]:
%%git_commit

training = pd.read_csv('../archive/Combined_Flights_2018.csv').sample(1000000)
testing = pd.read_csv('../archive/Combined_Flights_2019.csv').sample(1000000)


In [23]:
%%git_commit

training = training.dropna()
testing = testing.dropna()





In [24]:
%%git_commit

dropped_column = ['Flight_Number_Marketing_Airline', 'Flight_Number_Operating_Airline', 'DepTime', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'ActualElapsedTime', 'DepDelayMinutes', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrDelay', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'DistanceGroup', 'DivAirportLandings']




In [25]:
%%git_commit

for column in dropped_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)


In [17]:
%%git_commit


training['CRSDepTime'].unique()

array([1025, 1555, 1630, ...,  438,  317,  441])

# Sort by Appropriate Data

'Month', 'DayofMonth', 'CRSDepTime' -> not actually completely information free, but good enough

In [26]:
%%git_commit
training = training.sort_values(['Month', 'DayofMonth', 'CRSDepTime'])
testing = testing.sort_values(['Month', 'DayofMonth', 'CRSDepTime'])


In [28]:
%%git_commit

training.keys()

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'CRSElapsedTime', 'Distance', 'Year', 'Quarter', 'Month',
       'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline',
       'IATA_Code_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'OriginAirportID', 'OriginAirportSeqID',
       'OriginCityMarketID', 'OriginCityName', 'OriginState',
       'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID',
       'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState',
       'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'CRSArrTime'],
      dtype='object')

# One Hot Encode


In [29]:
%%git_commit

d2_column = ['FlightDate', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners' ,  'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'OriginAirportSeqID', 'DestAirportSeqID', 'CRSArrTime', 'DestStateName', 'DestState', 'DestCityName', 'OriginStateName', 'OriginState', 'OriginCityName','Year',  'Diverted', 'Cancelled', 'Tail_Number', 'Quarter',  'DayOfWeek', ]


In [30]:
%%git_commit

for column in d2_column:
    training = training.drop(column, axis = 1)
    testing = testing.drop(column, axis = 1)
    

In [ ]:
%%git_commit


# Model Training



In [ ]:
%%git_commit


# Present to Students


In [ ]:
%%git_commit
